# 🤖🤖 Multi-Agent Systems - Interactive Demo

**Welcome to Multi-Agent Collaboration!**

So far, we've built single agents that reason and act. But what happens when a task is too complex for one agent?

Just like in a company, you don't have one person do everything. You have specialists:
- A **researcher** who finds information
- A **writer** who creates content  
- A **reviewer** who checks quality
- A **manager** who coordinates them all

This demo shows you how to build **multi-agent systems** where specialized agents work together to solve complex problems.

## 📚 What This Demo Covers

### **1. Agent Roles & Specialization**
Create agents with specific expertise and tools.

### **2. Supervisor Pattern**
A manager agent that routes tasks to specialist agents.

### **3. Sequential Workflow**
Agents work in a pipeline, passing results along.

### **4. Parallel Agents**
Multiple agents work simultaneously on different subtasks.

### **5. Collaborative Agents**
Agents can communicate and build on each other's work.

### **6. Hierarchical Teams**
Teams of agents organized in layers (manager → team leads → workers).

---

**Real-world applications:**
- Research assistants (searcher + analyzer + summarizer)
- Content creation (researcher + writer + editor)
- Code development (architect + coder + tester)
- Customer service (classifier + specialist handlers)

In [ ]:
from dotenv import load_dotenv
load_dotenv()

## 1) Creating Specialized Agents

**The Concept:**
Each agent has:
- **Role**: What they specialize in
- **Tools**: Functions they can use
- **System Prompt**: Instructions defining their expertise

Think of it like hiring specialists rather than generalists.

### Example: Research Team
Let's create:
1. **Researcher**: Searches and gathers information
2. **Analyst**: Analyzes data and draws conclusions
3. **Writer**: Creates reports from findings

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage

llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Tools for different agents
@tool
def web_search(query: str) -> str:
    """Search the web for information."""
    # In production, this would use a real search API
    return f"Search results for '{query}': [Simulated web results about {query}]"

@tool  
def analyze_data(data: str) -> str:
    """Analyze data and provide insights."""
    return f"Analysis of data: [Simulated analysis insights]"

@tool
def format_report(content: str) -> str:
    """Format content into a professional report."""
    return f"# Professional Report\n\n{content}\n\n---\nGenerated Report"

# Agent 1: Researcher
researcher_prompt = """You are a research specialist.
Your job is to find and gather information using web searches.
Be thorough and cite sources."""

researcher_agent = ChatPromptTemplate.from_messages([
    ("system", researcher_prompt),
    ("human", "{input}")
]) | llm.bind_tools([web_search])

# Agent 2: Analyst
analyst_prompt = """You are a data analyst.
Your job is to analyze information and draw meaningful conclusions.
Focus on insights and patterns."""

analyst_agent = ChatPromptTemplate.from_messages([
    ("system", analyst_prompt),
    ("human", "{input}")
]) | llm.bind_tools([analyze_data])

# Agent 3: Writer
writer_prompt = """You are a professional report writer.
Your job is to create clear, well-structured reports.
Make content accessible and engaging."""

writer_agent = ChatPromptTemplate.from_messages([
    ("system", writer_prompt),
    ("human", "{input}")
]) | llm.bind_tools([format_report])

print("✅ Created 3 specialized agents")
print("  - Researcher (web_search tool)")
print("  - Analyst (analyze_data tool)")
print("  - Writer (format_report tool)")


## 2) Sequential Agent Workflow

**The Pattern:**
Agents work in a pipeline, each building on the previous agent's output.

```
User Query → Researcher → Analyst → Writer → Final Report
```

**Real-world example:**
Creating a market research report:
1. Researcher gathers market data
2. Analyst identifies trends
3. Writer creates executive summary

### Code Context
We use LangGraph to create a linear workflow where each agent processes the output of the previous one.

In [ ]:
from langgraph.graph import StateGraph, END, START
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage

# Define the state that flows through agents
class ResearchState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    topic: str
    research_findings: str
    analysis: str
    final_report: str

# Node functions
def research_node(state: ResearchState) -> ResearchState:
    """Researcher gathers information."""
    topic = state["topic"]
    
    # Simulate research
    findings = f"Research findings on {topic}:\n- Key fact 1\n- Key fact 2\n- Key fact 3"
    
    message = AIMessage(content=f"Completed research on: {topic}")
    
    return {
        "messages": [message],
        "research_findings": findings
    }

def analyze_node(state: ResearchState) -> ResearchState:
    """Analyst processes research findings."""
    findings = state["research_findings"]
    
    # Simulate analysis
    analysis = f"Analysis:\nBased on the research, key insights are:\n1. Insight A\n2. Insight B"
    
    message = AIMessage(content="Completed analysis")
    
    return {
        "messages": [message],
        "analysis": analysis
    }

def write_node(state: ResearchState) -> ResearchState:
    """Writer creates final report."""
    topic = state["topic"]
    findings = state["research_findings"]
    analysis = state["analysis"]
    
    report = f"""# Report: {topic}

## Research Findings
{findings}

## Analysis
{analysis}

## Conclusion
Based on our research and analysis, we recommend...
"""
    
    message = AIMessage(content="Report completed")
    
    return {
        "messages": [message],
        "final_report": report
    }

# Build the sequential workflow
workflow = StateGraph(ResearchState)

workflow.add_node("researcher", research_node)
workflow.add_node("analyst", analyze_node)
workflow.add_node("writer", write_node)

# Linear flow
workflow.add_edge(START, "researcher")
workflow.add_edge("researcher", "analyst")
workflow.add_edge("analyst", "writer")
workflow.add_edge("writer", END)

sequential_app = workflow.compile()

# Test it
result = sequential_app.invoke({
    "topic": "AI trends in 2025",
    "messages": [],
    "research_findings": "",
    "analysis": "",
    "final_report": ""
})

print("="*80)
print("SEQUENTIAL MULTI-AGENT WORKFLOW")
print("="*80)
print(result["final_report"])

## 3) Supervisor Pattern (Router Agent)

**The Problem:**
What if different queries need different specialists? You don't want to run all agents for every query.

**The Solution:**
Create a **supervisor agent** that:
1. Analyzes the user's request
2. Decides which specialist to route it to
3. Returns the specialist's response

```
                      ┌─→ Code Expert
User → Supervisor ─────→ Math Expert  
                      └─→ Writing Expert
```

### Code Context
The supervisor uses structured output to decide which agent should handle the task.

In [ ]:
from pydantic import BaseModel, Field
from enum import Enum

# Define available agents
class AgentType(str, Enum):
    RESEARCHER = "researcher"
    CODER = "coder"
    WRITER = "writer"
    FINISH = "finish"

class SupervisorDecision(BaseModel):
    """Decision on which agent should handle the task."""
    next_agent: AgentType = Field(description="The agent that should handle this task")
    reasoning: str = Field(description="Why this agent was chosen")

# Supervisor agent
supervisor_prompt = """You are a supervisor managing a team of specialists:

- researcher: Finds information, searches the web, gathers data
- coder: Writes code, debugs, implements solutions
- writer: Creates documents, reports, summaries
- finish: Task is complete, no more work needed

Given the user's request, decide which agent should handle it next.

Current request: {input}
Work completed so far: {context}
"""

supervisor_llm = llm.with_structured_output(SupervisorDecision)
supervisor = ChatPromptTemplate.from_template(supervisor_prompt) | supervisor_llm

# Test the supervisor
decision = supervisor.invoke({
    "input": "Write a Python function to calculate fibonacci numbers",
    "context": "No work done yet"
})

print(f"Supervisor Decision: Route to '{decision.next_agent}'")
print(f"Reasoning: {decision.reasoning}")

## 4) Complete Supervisor Multi-Agent System

Now let's build the full system with the supervisor routing tasks to specialists.

In [ ]:
from typing import Literal

class SupervisorState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    next_agent: str

# Specialist agent nodes
def researcher_node(state: SupervisorState) -> SupervisorState:
    last_message = state["messages"][-1]
    response = f"🔍 Researcher: I found information about '{last_message.content}'"
    return {"messages": [AIMessage(content=response)]}

def coder_node(state: SupervisorState) -> SupervisorState:
    last_message = state["messages"][-1]
    response = f"💻 Coder: Here's code for '{last_message.content}':\n```python\ndef solution():\n    pass\n```"
    return {"messages": [AIMessage(content=response)]}

def writer_node(state: SupervisorState) -> SupervisorState:
    last_message = state["messages"][-1]
    response = f"✍️ Writer: I've created content for '{last_message.content}'"
    return {"messages": [AIMessage(content=response)]}

def supervisor_node(state: SupervisorState) -> SupervisorState:
    messages = state["messages"]
    
    # Get last user message
    user_message = next((m for m in reversed(messages) if isinstance(m, HumanMessage)), None)
    
    if not user_message:
        return {"next_agent": "finish"}
    
    # Build context
    context = "\n".join([m.content for m in messages[-3:]])
    
    # Make decision
    decision = supervisor.invoke({
        "input": user_message.content,
        "context": context
    })
    
    return {"next_agent": decision.next_agent.value}

# Router function
def router(state: SupervisorState) -> str:
    return state.get("next_agent", "supervisor")

# Build graph
supervisor_graph = StateGraph(SupervisorState)

supervisor_graph.add_node("supervisor", supervisor_node)
supervisor_graph.add_node("researcher", researcher_node)
supervisor_graph.add_node("coder", coder_node)
supervisor_graph.add_node("writer", writer_node)

supervisor_graph.add_edge(START, "supervisor")
supervisor_graph.add_conditional_edges(
    "supervisor",
    router,
    {
        "researcher": "researcher",
        "coder": "coder",
        "writer": "writer",
        "finish": END
    }
)

# After specialist, go back to supervisor
supervisor_graph.add_edge("researcher", "supervisor")
supervisor_graph.add_edge("coder", "supervisor")
supervisor_graph.add_edge("writer", "supervisor")

supervisor_app = supervisor_graph.compile()

# Test with different queries
test_queries = [
    "Find information about LangChain",
    "Write a function to sort a list",
    "Create a summary of our findings"
]

print("="*80)
print("SUPERVISOR MULTI-AGENT SYSTEM")
print("="*80)

for query in test_queries:
    print(f"\n📝 Query: {query}")
    result = supervisor_app.invoke({
        "messages": [HumanMessage(content=query)],
        "next_agent": ""
    })
    print(f"Response: {result['messages'][-1].content}")
    print("-" * 80)

## 5) Collaborative Agents (Agent Network)

**The Pattern:**
Agents can communicate with each other directly, building a network rather than a hierarchy.

**Example: Code Review Team**
```
Developer → writes code
     ↓
Reviewer → checks code → [Issues found?]
     ↓ Yes                     ↓ No
Developer (fixes) ←────────→ Approved
```

### Code Context
We create a loop where agents iterate until quality standards are met.

In [ ]:
class CodeReviewState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    code: str
    issues: list[str]
    approved: bool
    iteration: int

# Developer agent
def developer_node(state: CodeReviewState) -> CodeReviewState:
    iteration = state.get("iteration", 0)
    issues = state.get("issues", [])
    
    if iteration == 0:
        # Initial code
        code = """def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)"""
        msg = "Initial code written"
    else:
        # Fix issues
        code = """def fibonacci(n):
    if n < 0:
        raise ValueError("n must be non-negative")
    if n <= 1:
        return n
    # Use memoization for efficiency
    cache = {0: 0, 1: 1}
    for i in range(2, n + 1):
        cache[i] = cache[i-1] + cache[i-2]
    return cache[n]"""
        msg = f"Fixed issues: {', '.join(issues)}"
    
    return {
        "messages": [AIMessage(content=msg)],
        "code": code,
        "iteration": iteration + 1
    }

# Reviewer agent
def reviewer_node(state: CodeReviewState) -> CodeReviewState:
    code = state["code"]
    iteration = state["iteration"]
    
    if iteration == 1:
        # First review - find issues
        issues = [
            "Missing input validation",
            "Inefficient recursive approach",
            "No docstring"
        ]
        approved = False
        msg = f"Found {len(issues)} issues"
    else:
        # Second review - approve
        issues = []
        approved = True
        msg = "✅ Code approved!"
    
    return {
        "messages": [AIMessage(content=msg)],
        "issues": issues,
        "approved": approved
    }

# Router
def review_router(state: CodeReviewState) -> str:
    if state.get("approved", False):
        return "end"
    else:
        return "developer"

# Build collaborative graph
collab_graph = StateGraph(CodeReviewState)

collab_graph.add_node("developer", developer_node)
collab_graph.add_node("reviewer", reviewer_node)

collab_graph.add_edge(START, "developer")
collab_graph.add_edge("developer", "reviewer")
collab_graph.add_conditional_edges(
    "reviewer",
    review_router,
    {"developer": "developer", "end": END}
)

collab_app = collab_graph.compile()

# Run the collaborative review process
print("="*80)
print("COLLABORATIVE AGENT NETWORK (Code Review)")
print("="*80)

result = collab_app.invoke({
    "messages": [],
    "code": "",
    "issues": [],
    "approved": False,
    "iteration": 0
})

print("\nProcess:")
for msg in result["messages"]:
    print(f"  - {msg.content}")

print("\nFinal Code:")
print(result["code"])
print(f"\nApproved: {result['approved']}")

## 6) Parallel Agent Execution

**The Pattern:**
When subtasks are independent, run agents in parallel for speed.

**Example: Competitive Analysis**
```
                    ┌─→ Competitor A Analyst
Start → Delegate ───┼─→ Competitor B Analyst → Combine → Report
                    └─→ Competitor C Analyst
```

All three analysts work simultaneously, then results are combined.

### Code Context
We use LangGraph's ability to add multiple edges from one node to execute agents in parallel.

In [ ]:
from functools import reduce

class ParallelState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    results: Annotated[list[str], operator.add]

# Parallel worker agents
def analyze_competitor_a(state: ParallelState) -> ParallelState:
    import time
    time.sleep(0.5)  # Simulate work
    return {"results": ["Competitor A: Market share 25%, growing"]}

def analyze_competitor_b(state: ParallelState) -> ParallelState:
    import time
    time.sleep(0.5)  # Simulate work  
    return {"results": ["Competitor B: Market share 30%, stable"]}

def analyze_competitor_c(state: ParallelState) -> ParallelState:
    import time
    time.sleep(0.5)  # Simulate work
    return {"results": ["Competitor C: Market share 15%, declining"]}

def combine_results(state: ParallelState) -> ParallelState:
    results = state["results"]
    summary = "Competitive Analysis Summary:\n" + "\n".join(f"  • {r}" for r in results)
    return {"messages": [AIMessage(content=summary)]}

# Note: LangGraph's current implementation runs nodes sequentially by default
# For true parallelism in production, you'd use:
# - LangGraph Cloud with parallel execution
# - Async implementations
# - Manual threading/multiprocessing

print("="*80)
print("PARALLEL AGENT PATTERN (Conceptual)")
print("="*80)
print("\nIn production, these agents would run simultaneously:")
print("  1. Competitor A Analyst")
print("  2. Competitor B Analyst")  
print("  3. Competitor C Analyst")
print("\nThen results would be combined into a final report.")
print("\nThis pattern is ideal for:")
print("  ✅ Independent subtasks")
print("  ✅ I/O-bound operations (API calls, web scraping)")
print("  ✅ Faster total execution time")

## 📊 Comparison of Multi-Agent Patterns

| Pattern | Use Case | Complexity | Coordination |
|---------|----------|------------|-------------|
| **Sequential** | Pipeline workflows | Low | None (linear) |
| **Supervisor** | Task routing to specialists | Medium | Centralized |
| **Collaborative** | Iterative improvement | High | Peer-to-peer |
| **Parallel** | Independent subtasks | Medium | Fan-out/fan-in |
| **Hierarchical** | Large organizations | Very High | Multi-level |

## 🎯 When to Use Multi-Agent Systems

**Use multi-agent when:**
- ✅ Task requires different expertise areas
- ✅ Subtasks can benefit from specialization
- ✅ Need quality checks (reviewer agents)
- ✅ Parallel execution speeds things up
- ✅ Complex workflows with branching logic

**Stick with single agent when:**
- ❌ Task is simple and straightforward  
- ❌ No clear specialization needed
- ❌ Coordination overhead > benefits
- ❌ All steps need same context

## 💡 Production Best Practices

### 1. **Clear Agent Boundaries**
Each agent should have a well-defined role and responsibility.

### 2. **Structured Communication**
Use Pydantic models for agent-to-agent messages to ensure type safety.

### 3. **Error Handling**
What happens if one agent fails? Have fallback strategies.

### 4. **Observability**
Track which agent did what using LangSmith tracing.

### 5. **Cost Management**
More agents = more LLM calls = higher cost. Monitor and optimize.

### 6. **State Management**
Keep shared state minimal. Only include what agents truly need.

## 📚 Real-World Examples

### Customer Support System
```
Classifier → [Bug|Feature|Billing]
              ↓       ↓        ↓
           Engineer Product Finance
```

### Content Creation Pipeline
```
Researcher → Outliner → Writer → Editor → Publisher
```

### Software Development Team
```
PM (defines requirements)
 ↓
Architect (designs system)
 ↓
Developer (writes code) ←→ Reviewer (checks quality)
 ↓
Tester (validates)
```

## 🔗 Additional Resources

- [LangGraph Multi-Agent Tutorial](https://langchain-ai.github.io/langgraph/tutorials/multi_agent/)
- [Agent Collaboration Patterns](https://blog.langchain.dev/planning-agents/)
- [AutoGen Framework](https://microsoft.github.io/autogen/) - Alternative multi-agent framework
- [CrewAI](https://www.crewai.com/) - Role-based agent orchestration

## ✅ Key Takeaways

1. **Specialization improves quality** - Expert agents outperform generalists
2. **Choose the right pattern** - Sequential, supervisor, or collaborative
3. **Manage complexity** - More agents = more coordination overhead
4. **Test thoroughly** - Multi-agent systems have more failure modes
5. **Monitor costs** - Multiple LLM calls add up quickly
6. **Use in production carefully** - Start simple, add agents as needed